In [ ]:
import sys

sys.version

In [ ]:
from functools import wraps

In [ ]:
class Property(property):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __new__(cls, *args, **kwargs):
        print(cls, args, kwargs)
        obj = super().__new__(cls)
        print(obj)
        return obj

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(self, fget, fset, fget, doc)
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
from time import sleep
from abc import ABC, ABCMeta, abstractmethod


def compute(obj, s):
    print(f"Computing {s} of {obj} ...", end="")
    sleep(3)
    print("DONE!")
    return "Phew, that was a lot of work!"


class MyMetaClass(ABCMeta):
    @property
    def expensive_metaclass_property(cls):
        """This may take a while to compute!"""
        return compute(cls, "metaclass property")


class MyBaseClass(ABC, metaclass=MyMetaClass):
    @classmethod
    def expensive_class_property(cls):
        """This may take a while to compute!"""
        return compute(cls, "class property")

    @Property
    def expensive_instance_property(self):
        """This may take a while to compute!"""
        return compute(self, "instance property")


class MyClass(MyBaseClass):
    """Some subclass of MyBaseClass"""


# help(MyClass)